In [1]:
from hmmfunc import MonoPhoneHMM
import os
import numpy as np

ModuleNotFoundError: No module named 'hmmfunc'

In [2]:
hmm_file = "../03gmm_hmm/exp/model_3state_2mix/10.hmm"
feat_scp = "../01compute_features/mfcc/train_small/feats.scp"
label_file = "../03gmm_hmm/exp/data/train_small/text_int"
align_file = "./exp/data/train_small/alignment"

In [10]:
class MonoPhoneHMM2(MonoPhoneHMM):
    def __init__(self):
        super().__init__()
    
    def state_alignment2(self, feat, label):
        self.calc_out_prob(feat, label)
        self.viterbi_decoding(label)
        viterbi_path = self.back_track()
        state_alignment = []
        for vp in viterbi_path:
            l = vp[0]
            p = label[l]
            s = vp[1]
            state = p * self.num_states + s
            state_alignment.append(state)
        return state_alignment

In [12]:
hmm = MonoPhoneHMM2()
hmm.load_hmm(hmm_file)

In [13]:
out_dir = os.path.dirname(align_file)
os.makedirs(out_dir, exist_ok=True)

In [14]:
label_list = {}
with open(label_file, mode="r") as f:
    for line in f:
        utt = line.split()[0]
        lab = line.split()[1:]
        lab = np.int64(lab)
        label_list[utt] = lab

In [15]:
with open(align_file, mode="w") as fa, open(feat_scp, mode="r") as fs:
    for line in fs:
        utt = line.split()[0]
        feat_file = line.split()[1]

        label = label_list[utt]
        feat = np.fromfile(feat_file, dtype=np.float32)
        # フレームバッチの形に変換
        feat = feat.reshape(-1, hmm.num_dims)

        alignment = hmm.state_alignment2(feat, label)
        # map: alignment内の各数値をstr型に変換
        alignment = " ".join(map(str, alignment))
        fa.write(f"{utt} {alignment}\n")


